# Dev: `fs`

In [ ]:
import os
from pathlib import Path

## `read_text()`

In [ ]:
def read_text(path, encoding='utf-8'):    
    """Read a text file and return its content as a string.
    """
    path = Path(path)
    try:
        # Open the text file in read mode
        with open(path, 'r', encoding=encoding) as file:
            # Read the entire file content into a string
            text_content = file.read()
            return text_content
    except FileNotFoundError:
        print(f"File was not found: `{path}`")
    except Exception:
        raise

In [44]:
read_text("not-exist.txt")

File was not found: `not-exist.txt`


In [46]:
read_text("text/hello.txt")
read_text(Path("text/hello.txt"))

'Hello world\nThis is secound line\n'

### HowTo

In [ ]:
with open("text/hello.txt", 'r', encoding='utf-8') as file:
    res1 = file.read()
    
res1

'Hello world\nThis is secound line\n'

In [6]:
with open("text/hello.txt", 'r', encoding='utf-8') as file:
    res2 = file.readlines()
    
res2

['Hello world\n', 'This is secound line\n']

## `write_text()`

In [101]:
def write_text(text, path, encoding="utf-8", recurse = False, append = False):
    """Write a text string to a text file.
    """
    path = Path(path)
    try:
        # Create intermediate directories if they don't exist
        if recurse and not path.parent.exists():
            path.parent.mkdir(parents=True, exist_ok=True)
            print(f"Created intermediate dir: {path.parent}")
        
        # Open the text file to write or append content
        with open(path, "a" if append else "w", encoding=encoding) as file:
            file.write(text)
            if append:
                print(f"Text successfully append to {path}")
            else:
                print(f"Text successfully written to {path}")
        
    except (PermissionError, IsADirectoryError):
        print(f"Cannot write to file: `{path}`")
    except Exception:
        raise


In [96]:
write_text("hi", Path("text/hello-write.txt"), append=False)
write_text("hi2", Path("text/hello-write2.txt"), append=False)

Text successfully written to text/hello-write.txt
Text successfully written to text/hello-write2.txt


In [102]:
write_text("hi", Path("text/subdir2/hello-write.txt"), recurse=False)

FileNotFoundError: [Errno 2] No such file or directory: 'text/subdir2/hello-write.txt'

In [105]:
write_text("hi", Path("text/subdir/sub/hello-write.txt"), recurse=True)

Created intermediate dir: text/subdir/sub
Text successfully written to text/subdir/sub/hello-write.txt
